In [4]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd
%cd /content/drive/My Drive/Independent Study/validation

# dataset = load_dataset("csv", data_files={'export_L_dataframe.csv'})
# df = pd.read_csv("trump_biden_headlines.csv")
# covid_df = pd.read_csv("covid_validation.csv")

Mounted at /content/drive/
/content/drive/My Drive/Independent Study/validation


In [ ]:
df.size

375390

In [ ]:
# Topic modelling
# 1) Covid
# 2) Economy
# 3) Gun-violence
# 4) Abortion
# 5) LGBTQ rights

covid_keywords = ["coronavirus", "covid", "vaccine", "covid19", "covid-19", "mask", "masks", "mandate", "immunity", "immunization", "vaccination", "cases", "recovery", "pandemic", "infection", "outbreak", "coronavirusoutbreak", "covidoutbreak", "virus", "social distance", "testing", "spread", "variant", "infection", "pfizer", "astrazeneca"]
gun_violence_keywords = ["gun", "guns", "violence", "distruction", "ammunition", "arms", "pistol", "gun-violence"]
economy_keywords = ["economy", "money", "market"]
abortion_keywords = ["abortion", "abortions", "abortion-rights", "pills", "pill"]
lgbtq_rights_keywords = ["gay", "lesbian", "trans", "lgbtq", "lgbtq+", "lesbians"]

In [ ]:
import nltk
nltk.download('all')

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Analyzer = SentimentIntensityAnalyzer()

Analyzer.polarity_scores('happy')['compound']

0.5719

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Analyzer = SentimentIntensityAnalyzer()

sentences = []
masked_sentences = []
publications = []
topics = []

def mask_word_in_sentence(sentence):
  # sentence = sentence.lower() 
  # sentence = sentence.replace("president", "")

  # sentence = sentence.replace("biden", "democrats")
  # sentence = sentence.replace("trump", "republicans")
  tokenized_sentence = nltk.word_tokenize(sentence)
  polarity_dict = {}
  pos_words = nltk.pos_tag(tokenized_sentence)
  topic = ""
  for word in pos_words:
    if word[1] == 'NNP' or word[1] == 'DT':
      tokenized_sentence.remove(word[0])
  for word in tokenized_sentence:
    if word == 'trump' or word =='biden' or word == 'joe' or word == 'donald' or word == ':':
      continue
    
    # if word in gun_violence_keywords:
    #   topic = "gun_violence" 
    # if word in economy_keywords:
    #   topic = "economy" 
    # if word in abortion_keywords:
    #   topic = "abortion" 
    # if word in lgbtq_rights_keywords:
    #   topic = "lgbtq" 
    if word in covid_keywords:
      topic = "covid"
    polarity_dict[word] = abs(Analyzer.polarity_scores(word)['compound'])

  polarity_dict = dict(sorted(polarity_dict.items(), key=lambda x: x[1], reverse = True))
  highest_valence_word = list(polarity_dict.keys())[0]
  masked_sentence = sentence.replace(highest_valence_word, "[MASK]")
  return sentence, masked_sentence, topic


for index, row in df.iterrows():
  sentence = str(row['title']).lower()
  # sentence = sentence.replace(":", " says")
  sentence = sentence.replace("'", "")
  sentence = sentence.replace("dr.", "")
  # print(sentence)
  # sentence = "Hello Rahil, You have to build a very good site, and I love visiting your site."
  tokenized_sentence = sent_tokenize(sentence)
 
  sentence, masked_sentence, topic = mask_word_in_sentence(sentence)
  if "trump" in sentence or "biden" in sentence:
    sentences.append(sentence)
    publications.append(row['publication'])
    masked_sentences.append(masked_sentence)
    topics.append(topic)


In [ ]:
len(masked_sentences)

62563

In [ ]:
validation_df = pd.DataFrame()
# validation_df['date'] = df['date']
validation_df['sentences'] = sentences
validation_df['masked_sentences'] = masked_sentences
validation_df['publications'] = publications
validation_df['topics'] = topics

In [ ]:
validation_df.head(18)

,sentences,masked_sentences,publications,topics
0,trump fires off tweets as us tackles coronavir...,trump fires off tweets as us tackles coronavir...,ABC News,covid
1,trump says he intends to reopen country in wee...,trump [MASK] he intends to reopen country in w...,ABC News,
2,trump considering loosening coronavirus guidel...,trump considering loosening coronavirus guidel...,ABC News,covid
3,"on a call with trump officials, police chief p...","on a call with trump officials, police chief p...",ABC News,covid
4,fauci says he cant stop trump from talking at ...,fauci says he cant [MASK] trump from talking a...,ABC News,
5,trump says he intends to reopen country in wee...,trump [MASK] he intends to reopen country in w...,ABC News,
6,"coronavirus government response updates: us, t...","[MASK] government response updates: us, trump ...",ABC News,covid
7,new york mayor blames trump for medical shortages,new york mayor [MASK] trump for medical shortages,Al Jazeera English,
8,"state, local officials to trump: lead or get o...","[MASK], local officials to trump: lead or get ...",Al Jazeera English,
9,trump defends calling coronavirus the chinese ...,trump [MASK] calling coronavirus the chinese v...,Al Jazeera English,covid


In [ ]:
dfs = [x for _, x in validation_df.groupby('topics')]
covid_df = dfs[1]

In [ ]:
len(dfs[1])

10109

In [ ]:
covid_df.to_csv (r'covid_validation.csv', index = False, header=True)

## Topic modeling - BERTopic

In [ ]:
# !pip install bertopic
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
 
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

topic_model = BERTopic()
# topics, probs = topic_model.fit_transform(docs)

# topics, probs = topic_model.fit_transform(validation_df['sentences'])
topics, probs = topic_model.fit_transform(vdf)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
vdf = []
for doc in processed_docs:
  doc = " ".join(doc)
  vdf.append(doc)


In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,2974,-1_covid_coronavirus_trump_say
1,0,647,0_pandem_recoveri_econom_amid
2,1,264,1_brief_coronavirus_respons_daili
3,2,250,2_virus_worst_surg_away
4,3,244,3_mask_wear_face_debat


In [ ]:
topic_model.get_topic(5)

[('vaccin', 0.05993315723915398),
 ('distribut', 0.034617128980766035),
 ('biden', 0.03245872085938249),
 ('dose', 0.026212775485866546),
 ('pfizer', 0.022766836715057106),
 ('rollout', 0.021847392879060925),
 ('plan', 0.02071988957435857),
 ('receiv', 0.020222048517161403),
 ('amazon', 0.018622341326936402),
 ('vow', 0.016966540237932693)]

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=6)

## Validation df saving

In [ ]:
validation_df.head(18)

,sentence,masked_sentences,publication
0,trump fires off tweets as us tackles coronavir...,trump fires off tweets as us tackles coronavir...,ABC News
1,trump says he intends to reopen country in wee...,trump [MASK] he intends to reopen country in w...,ABC News
2,trump considering loosening coronavirus guidel...,trump considering loosening coronavirus guidel...,ABC News
3,"on a call with trump officials, police chief p...","on a call with trump officials, police chief p...",ABC News
4,fauci says he can't stop trump from talking at...,fauci says he can't [MASK] trump from talking ...,ABC News
5,trump says he intends to reopen country in wee...,trump [MASK] he intends to reopen country in w...,ABC News
6,"coronavirus government response updates: us, t...","[MASK] government response updates: us, trump ...",ABC News
7,new york mayor blames trump for medical shortages,new york mayor [MASK] trump for medical shortages,Al Jazeera English
8,"state, local officials to trump: 'lead or get ...","[MASK], local officials to trump: 'lead or get...",Al Jazeera English
9,trump defends calling coronavirus the 'chinese...,trump [MASK] calling coronavirus the 'chinese ...,Al Jazeera English


In [ ]:
print(validation_df['sentence'][38], validation_df['masked_sentences'][38])

trump's favorite fox news hosts are pushing him to prioritize the economy over social distancing amid the coronavirus pandemic trump's [MASK] fox news hosts are pushing him to prioritize the economy over social distancing amid the coronavirus pandemic


In [ ]:
validation_df.to_csv (r'validation_results_65k_wo_target_mask.csv', index = False, header=True)

In [ ]:
import nltk
text = "Hello Rahil, You have to build a very good site, and I love visiting your site."
sentence = nltk.sent_tokenize(text)
for sent in sentence:
	 print(nltk.pos_tag(nltk.word_tokenize(sent)))

[('Hello', 'NNP'), ('Rahil', 'NNP'), (',', ','), ('You', 'PRP'), ('have', 'VBP'), ('to', 'TO'), ('build', 'VB'), ('a', 'DT'), ('very', 'RB'), ('good', 'JJ'), ('site', 'NN'), (',', ','), ('and', 'CC'), ('I', 'PRP'), ('love', 'VBP'), ('visiting', 'VBG'), ('your', 'PRP$'), ('site', 'NN'), ('.', '.')]


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

text = "Hello Rahil, You have to build a very good site, and I love visiting your site."


# sent_tokenize is one of instances of
# PunktSentenceTokenizer from the nltk.tokenize.punkt module

tokenized = sent_tokenize(text)
print(tokenized)
for i in tokenized:

	
	# Word tokenizers is used to find the words
	# and punctuation in a string
	wordsList = nltk.word_tokenize(i)

	# removing stop words from wordList
	wordsList = [w for w in wordsList ]

	# Using a Tagger. Which is part-of-speech
	# tagger or POS-tagger.
	tagged = nltk.pos_tag(wordsList)
	print(tagged)


['Hello Rahil, You have to build a very good site, and I love visiting your site.']
[('Hello', 'NNP'), ('Rahil', 'NNP'), (',', ','), ('You', 'PRP'), ('have', 'VBP'), ('to', 'TO'), ('build', 'VB'), ('a', 'DT'), ('very', 'RB'), ('good', 'JJ'), ('site', 'NN'), (',', ','), ('and', 'CC'), ('I', 'PRP'), ('love', 'VBP'), ('visiting', 'VBG'), ('your', 'PRP$'), ('site', 'NN'), ('.', '.')]


## Covid Grouping

In [1]:
cases_deaths_keywords = ['surge', 'high', 'case', 'wave', 'cases', 'deaths', 'death']
economy_education_impact_keywords = ['student', 'child', 'shut', 'life', 'work', 'year', 'business', 'businesses', 'shutting', 'market', 'economy', 'worker', 'employer', 'employee', 'unemployment', 'unemployed']
politics_keywords = ['election', 'government', 'national', 'announce', 'official','live', 'host', 'american', 'response', 'crisis', 'plan', 'president', 'district', 'state', 'deal', 'conduct', 'issues', 'fundraising', 'politicizing', 'governor']
preventive_measures_keywords = ['plan', 'stop', 'urge', 'break', 'stop', 'spread', 'measure', 'extend', 'ease', 'lift', 'restriction', 'lockdown']
virus_spreading_keywords = ['outbreak', 'virus', 'spread', 'warn', 'global', 'emergency', 'infect', 'infection', 'risk', 'spreading', 'rise', 'rising']
vaccine_keywords = ['vaccine', 'pfizer', 'dose', 'dosage', 'receive', 'astrazeneca', 'supply', 'jab', 'drug']

In [2]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [5]:
from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

covid_df = pd.read_csv("covid_validation.csv")

Analyzer = SentimentIntensityAnalyzer()

sentences = []
masked_sentences = []
publications = []
topics = []

def check_word_similarity(word1, word2):
  word1_wordnet = wordnet.synsets(word1)
  word2_wordnet = wordnet.synsets(word2)
  score = 0
  if word1_wordnet and word2_wordnet:
    score = word1_wordnet[0].wup_similarity(word2_wordnet[0])
  return score

def check_word_similarity_for_entire_list(word, wordList):
  for keyword in wordList:
      score = check_word_similarity(keyword, word)
      if score > 0.92:
        return True
  return False



def mask_word_in_sentence(sentence):
  # sentence = sentence.lower() 
  # sentence = sentence.replace("president", "")

  # sentence = sentence.replace("biden", "democrats")
  # sentence = sentence.replace("trump", "republicans")
  tokenized_sentence = nltk.word_tokenize(sentence)
  polarity_dict = {}
  pos_words = nltk.pos_tag(tokenized_sentence)
  topic = ""
  if tokenized_sentence[1] == ':':
    tokenized_sentence = tokenized_sentence[2:]
  for word in pos_words:
    if word[1] == 'NNP' or word[1] == 'DT':
      tokenized_sentence.remove(word[0])
  for word in tokenized_sentence:
    if word == 'trump' or word =='biden' or word == 'joe' or word == 'donald' or word[-1] == ':':
      continue
    if word[-1] == ':':
      word = word[:len(word) - 1]
    if word in cases_deaths_keywords or check_word_similarity_for_entire_list(word, cases_deaths_keywords):
      topic = "cases_deaths" 
    if word in politics_keywords or check_word_similarity_for_entire_list(word, politics_keywords):
      topic = "politics" 
    if word in preventive_measures_keywords or check_word_similarity_for_entire_list(word, preventive_measures_keywords):
      topic = "preventive_measures" 
    if word in virus_spreading_keywords or check_word_similarity_for_entire_list(word, virus_spreading_keywords):
      topic = "virus_spreading"
    if word in economy_education_impact_keywords or check_word_similarity_for_entire_list(word, economy_education_impact_keywords):
      topic = "economy_education_impact" 
    if word in vaccine_keywords or check_word_similarity_for_entire_list(word, vaccine_keywords):
      topic = "vaccine"
    polarity_dict[word] = abs(Analyzer.polarity_scores(word)['compound'])

  polarity_dict = dict(sorted(polarity_dict.items(), key=lambda x: x[1], reverse = True))
  highest_valence_word = list(polarity_dict.keys())[0]
  masked_sentence = sentence.replace(highest_valence_word, "[MASK]")
  return sentence, masked_sentence, topic


for index, row in covid_df.iterrows():
  sentence = str(row['sentences']).lower()
  # sentence = sentence.replace(":", " says")
  sentence = sentence.replace("'", "")
  sentence = sentence.replace("dr.", "")
  # sentence = "Hello Rahil, You have to build a very good site, and I love visiting your site."
  tokenized_sentence = sent_tokenize(sentence)
 
  sentence, masked_sentence, topic = mask_word_in_sentence(sentence)
  if "trump" in sentence or "biden" in sentence:
    sentences.append(sentence)
    publications.append(row['publications'])
    masked_sentences.append(masked_sentence)
    topics.append(topic)


In [6]:
covid_validation_df = pd.DataFrame()
# validation_df['date'] = df['date']
covid_validation_df['sentences'] = sentences
covid_validation_df['masked_sentences'] = masked_sentences
covid_validation_df['publications '] = publications
covid_validation_df['topics'] = topics

In [7]:
covid_df_grouped = [x for _, x in covid_validation_df.groupby('topics')]

In [8]:
covid_validation_df.size

40436

In [9]:
len(covid_df_grouped)

7

In [ ]:
# Number of sentences in each group
# cases_deaths = 524
# economy_education_impact = 289
# politics = 1672
# preventive_measures = 294
# virus_spreading = 915
# vaccine = 294

In [10]:
len(covid_df_grouped[6])

975

In [11]:
covid_validation_df.to_csv (r'covid_grouped_validation.csv', index = False, header=True)

## Topic Modeling - LDA

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)


In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
import pandas as pd
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
'''
Preview a document after preprocessing
'''
document_num = 50
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replac']


In [ ]:
processed_docs = []

for doc in df['sentences']:
    processed_docs.append(preprocess(doc))

In [ ]:
print(len(processed_docs))

10109


In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 coronavirus
1 fire
2 guidelin
3 includ
4 messag
5 mitig
6 rollback
7 spread
8 support
9 tackl
10 trump


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 20 ("govern") appears 1 time.
Word 28 ("boost") appears 1 time.
Word 76 ("alli") appears 1 time.
Word 77 ("great") appears 1 time.
Word 78 ("market") appears 1 time.
Word 79 ("rule") appears 1 time.
Word 80 ("warn") appears 1 time.


In [ ]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.069*"relief" + 0.038*"sign" + 0.033*"check" + 0.030*"stimulus" + 0.025*"claim" + 0.025*"forc" + 0.022*"task" + 0.022*"congress" + 0.021*"fact" + 0.020*"packag"


Topic: 1 
Words: 0.117*"test" + 0.060*"posit" + 0.055*"donald" + 0.045*"elect" + 0.036*"presid" + 0.024*"diagnosi" + 0.020*"melania" + 0.019*"campaign" + 0.018*"updat" + 0.015*"recoveri"


Topic: 2 
Words: 0.113*"mask" + 0.048*"wear" + 0.021*"mandat" + 0.021*"face" + 0.020*"american" + 0.020*"order" + 0.020*"nation" + 0.018*"reopen" + 0.016*"debat" + 0.016*"virus"


Topic: 3 
Words: 0.138*"vaccin" + 0.041*"administr" + 0.033*"plan" + 0.021*"distribut" + 0.019*"know" + 0.019*"dose" + 0.019*"million" + 0.016*"thing" + 0.014*"harri" + 0.014*"speed"


Topic: 4 
Words: 0.026*"fight" + 0.023*"state" + 0.022*"china" + 0.022*"blame" + 0.019*"media" + 0.019*"virus" + 0.018*"administr" + 0.016*"court" + 0.016*"fauci" + 0.015*"crisi"


Topic: 5 
Words: 0.096*"respons" + 0.027*"fauci" + 0.027*"hous" + 0.026*"amid" + 0.0

## Similarity between words grouping

In [21]:
import nltk
# nltk.download('all')
from nltk.corpus import wordnet

word1 = wordnet.synsets('day')
word2 = wordnet.synsets('tomorrow')
score = word1[0].wup_similarity(word2[0])
print(score)

0.9090909090909091


In [ ]:
# !pip install spacy
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
  
print("Enter two space-separated words")
words = input()

tokens = nlp(words)
print(tokens)  
for token in tokens:
    # Printing the following attributes of each token.
    # text: the word string, has_vector: if it contains
    # a vector representation in the model, 
    # vector_norm: the algebraic norm of the vector,
    # is_oov: if the word is out of vocabulary.
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)
  
token1, token2 = tokens[0], tokens[1]
  
print("Similarity:", token1.similarity(token2))

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


Enter two space-separated words
runs ran
runs ran
runs True 9.515867 True
ran True 8.208196 True
Similarity: -0.04574021324515343


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2022-11-10 04:04:29--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.179.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.179.165|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-11-10 04:04:30 ERROR 404: Not Found.



In [ ]:
!pip install gensim
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
print('Imported Successfully!')

Imported Successfully!


In [6]:
# code for Glove word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

x = {'text', 'the', 'leader', 'prime',
	'natural', 'language'}

# create the dict.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

# number of unique words in dict.
print("Number of unique words in dictionary=",
	len(tokenizer.word_index))
print("Dictionary is = ", tokenizer.word_index)

# download glove and unzip it in Notebook.
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

# vocab: 'the': 1, mapping of words with
# integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index,
						embedding_dim):
	vocab_size = len(word_index) + 1
	
	# Adding again 1 because of reserved 0 index
	embedding_matrix_vocab = np.zeros((vocab_size,
									embedding_dim))

	with open(filepath, encoding="utf8") as f:
		for line in f:
			word, *vector = line.split()
			if word in word_index:
				idx = word_index[word]
				embedding_matrix_vocab[idx] = np.array(
					vector, dtype=np.float32)[:embedding_dim]

	return embedding_matrix_vocab


# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
	'./glove.6B.50d.txt', tokenizer.word_index,
embedding_dim)
print(embedding_matrix_vocab[1])
print("Dense vector for first word is => ",
	embedding_matrix_vocab[1])


Number of unique words in dictionary= 6
Dictionary is =  {'the': 1, 'text': 2, 'leader': 3, 'language': 4, 'prime': 5, 'natural': 6}
[ 4.18000013e-01  2.49679998e-01 -4.12420005e-01  1.21699996e-01
  3.45270008e-01 -4.44569997e-02 -4.96879995e-01 -1.78619996e-01
 -6.60229998e-04 -6.56599998e-01  2.78430015e-01 -1.47670001e-01
 -5.56770027e-01  1.46579996e-01 -9.50950012e-03  1.16579998e-02
  1.02040000e-01 -1.27920002e-01 -8.44299972e-01 -1.21809997e-01
 -1.68009996e-02 -3.32789987e-01 -1.55200005e-01 -2.31309995e-01
 -1.91809997e-01 -1.88230002e+00 -7.67459989e-01  9.90509987e-02
 -4.21249986e-01 -1.95260003e-01  4.00710011e+00 -1.85939997e-01
 -5.22870004e-01 -3.16810012e-01  5.92130003e-04  7.44489999e-03
  1.77780002e-01 -1.58969998e-01  1.20409997e-02 -5.42230010e-02
 -2.98709989e-01 -1.57490000e-01 -3.47579986e-01 -4.56370004e-02
 -4.42510009e-01  1.87849998e-01  2.78489990e-03 -1.84110001e-01
 -1.15139998e-01 -7.85809994e-01]
Dense vector for first word is =>  [ 4.18000013e-01  

In [ ]:
word_vectors = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

v_apple = word_vectors['banana']
v_mango = word_vectors['mango']
cosine_similarity([v_apple],[v_mango])

FileNotFoundError: ignored

## Compare two categories from two dfs

In [ ]:
covid_new_df = pd.read_csv("covid_validation.csv")
covid_old_df = pd.read_csv("covid_old_validation.csv")